In [21]:
isLocalServer = True
sys_path_to_be_added = '/notebook/personal/ksuchoi216/FaceID_model_rebuild/'
config_path = './configs/config.json'

%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
import os

def checkLocalServer(isLocalServer, sys_path_to_be_added=None):
    print('='*60)
    print(f'isLocalServer is {isLocalServer}')
    print('='*60)
    
    if isLocalServer is True:
        if sys_path_to_be_added not in sys.path:
            sys.path.insert(0, sys_path_to_be_added)
            os.chdir(sys_path_to_be_added)

checkLocalServer(isLocalServer, sys_path_to_be_added)

from utils import Config_Manager
cfgm = Config_Manager(config_path)
cfg = cfgm.get_cfg()
!pwd
print('='*60)

!nvidia-smi

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
isLocalServer is True
/notebook/personal/ksuchoi216/FaceID_model_rebuild
Thu Oct 13 18:49:40 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.129.06   Driver Version: 470.129.06   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:04:00.0 Off |                    0 |
| N/A   34C    P0    33W / 250W |   1333MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+------

In [22]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import KMeans
from sklearn import datasets, cluster

from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [23]:
import wandb
wandb.login()

True

In [24]:
import numpy as np
import os

def loadNumpy(source_path, file_name):
    file_path = os.path.join(source_path, file_name)
    
    if not os.path.exists(file_path):
        return print(f'The file doesnt exist: {file_path}')
    
    return np.load(file_path)


emb = loadNumpy('./data/extracted_np_97', 'face_emb.npy')
lb = loadNumpy('./data/extracted_np_97', 'face_lb.npy')

In [25]:
wandb.errors.term._show_warnings = False
X_train, X_test, y_train, y_test = train_test_split(emb, lb, test_size=0.3)

In [27]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
clf = make_pipeline(StandardScaler(), SVC(gamma='auto', probability=True))
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
y_proba = clf.predict_proba(X_test)


AttributeError: predict_proba is not available when  probability=False

In [26]:
run = wandb.init(project='faceid-experiment', name="svm")